In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np


# 데이터 로드
kdf = pd.read_csv('data/KINEMATIC.csv')
fdf = pd.read_csv('data/FORCE.csv')

kdf['trial'] = kdf['trial'].astype(int)
fdf['trial'] = fdf['trial'].astype(int)

# 스트림릿 사이드바 설정
unique_names = kdf['player'].unique()
selected_name = unique_names[0]
filtered_df_by_name = kdf[kdf['player'] == selected_name]
unique_dates = sorted(filtered_df_by_name['day'].unique())
selected_date = unique_dates[0]

filtered_df_by_name_datas = kdf[(kdf['player'] == selected_name) &
                                (kdf['day'] == selected_date)]
unique_trial = sorted(filtered_df_by_name_datas['trial'].unique())
selected_trial = unique_trial[0]

kine_filtered = kdf[(kdf['player'] == selected_name) & 
                    (kdf['day'] == selected_date) &
                    (kdf['trial'] == selected_trial)]

force_filtered = fdf[(fdf['player'] == selected_name) & 
                    (fdf['day'] == selected_date) &
                    (fdf['trial'] == selected_trial)]

kine_filtered.reset_index(inplace=True, drop=True)
force_filtered.reset_index(inplace=True, drop=True)

k_sr = 180
k_kh_time  = kine_filtered['kh_time'][0]
k_fc_time  = kine_filtered['fc_time'][0] - k_kh_time
k_mer_time = kine_filtered['mer_time'][0] - k_kh_time
k_br_time  = kine_filtered['br_time'][0] - k_kh_time
k_kh_time = kine_filtered['kh_time'][0] - k_kh_time
stride_length = round(kine_filtered['stride_length'][0])

f_sr = 1080
f_kh_time  = force_filtered['kh_time'][0] 
f_fc_time  = force_filtered['fc_time'][0]  - f_kh_time
f_mer_time = force_filtered['mer_time'][0] - f_kh_time
f_br_time  = force_filtered['br_time'][0]  - f_kh_time
f_kh_time = force_filtered['kh_time'][0]  - f_kh_time

k_df = kine_filtered.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
f_df = force_filtered.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

f_lead_peak_z_time = f_df['lead_peak_z'][0] - f_kh_time
f_rear_peak_z_time = np.where(f_df['REAR_FORCE_Z'] == f_df['REAR_FORCE_Z'].max())[0][0]
force_peak_time = round((f_lead_peak_z_time - f_rear_peak_z_time) / 1080 , 4)

f_rear_peak_y_time = np.where(f_df['REAR_FORCE_Y'] == f_df['REAR_FORCE_Y'].max())[0][0]
f_lead_min_y_time  = f_df['lead_valley_y'][0] - f_kh_time

k_df.drop(['kh_time','fc_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
f_df.drop(['kh_time','fc_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

k_len = len(k_df)
k_time = np.arange(0,k_len/k_sr, 1/k_sr)

f_len = len(f_df)
f_time = np.arange(0,f_len/f_sr, 1/f_sr)